# M62

Large-group monopolistic competition

|Markets  |   XI  |     X   |     N  |      Y  |      W   |  CONS |  ENTR    |
|---------|-------|---------|--------|---------|----------|-------|----------|
|    PX   |       |  100    |        |         |  -100    |       |          | 
|    CX   |  100  | -100    |        |         |          |       |          |
|    PY   |       |         |        |   100   |  -100    |       |          |
|    PF   |       |         |   20   |         |          |       |   -20    |  
|    PU   |       |         |        |         |   200    |  -200 |          |  
|    PW   |  -32  |         |   -8   |   -60   |          |   100 |          |  
|    PZ   |  -48  |         |  -12   |   -40   |          |   100 |          | 
|    MK   |  -20  |         |        |         |          |       |    20    | 

In [18]:
using MPSGE

In [19]:
M62 = MPSGEModel()

@parameters(M62, begin
    ENDOW, 1, (description = "Size index for the economy",)
    EP, 5, (description = "Elasticity of substitution among X varieties",)
end)


@sectors(M62, begin
	X,	(description = "Activity level for sector X",)
	Y,	(description = "Activity level for sector Y",)
	W,	(description = "Activity level for sector W (Hicksian welfare index)",)
	N,	(description = "Activity level for sector X fixed costs, no. of firms",)
	XI,	(description = "Activity level -- marginal cost of X",)
end)
@commodities(M62, begin
	PX,	(description = "Price index for commodity X (gross of markup)",)
	CX,	(description = "Marginal cost index for commodity X (net markup)",)
	PY,	(description = "Price index for commodity Y",)
	PW,	(description = "Price index for unskilled labor",)
	PZ,	(description = "Price index for skilled labor",)
	PF,	(description = "Unit price of inputs to fixed cost",)
	PU,	(description = "Price index for welfare (expenditure function)",)
end)
@consumers(M62, begin
	CONS,	(description = "Income level for consumer CONS",)
	ENTRE,	(description = "Entrepreneur (converts markup revenue to fixed cost)",)
end)
@auxiliaries(M62, begin
	XQADJ,	(description = "Quantity adjustment   (positive when X>1)",)
	XPADJ,	(description = "X output subsidy rate (positive when X>1)",)
end)

@production(M62, X, [s=1,t=0], begin
    @output(PX, 80, t, reference_price = 1.25, taxes = [Tax(CONS, XPADJ*-1)])
    @input(CX, 80, s, reference_price = 1.25)
end)

@production(M62, Y, [s=1,t=0], begin
    @output(PY, 100, t)
    @input(PW, 60, s)
    @input(PZ, 40, s)
end)

@production(M62, XI, [s=1,t=0], begin
    @output(CX, 80, t, taxes = [Tax(ENTRE,.2)])
    @input(PW, 32, s)
    @input(PZ, 48, s)
end)

@production(M62, N, [s=1,t=0], begin
    @output(PF, 20, t)
    @input(PZ, 12, s)
    @input(PW, 8, s)
end)

@production(M62, W, [s=1,t=0], begin
    @output(PU, 200, t)
    @input(PX, 80, s, reference_price = 1.25)
    @input(PY, 100, s)
end)

@demand(M62, CONS, begin
    @final_demand(PU, 200)
end, begin
    @endowment(PW, 100*ENDOW)
    @endowment(PZ, 100*ENDOW)
    @endowment(PX, 80*XQADJ)
end)

@demand(M62, ENTRE, begin
    @final_demand(PF, 20)
end, begin
end)

@aux_constraint(M62, XQADJ,
    XQADJ - (N^(1/(EP-1))*X - X)
)

@aux_constraint(M62, XPADJ, 
    XPADJ - (N^(1/(EP-1)) - 1)
)

ScalarAuxConstraint(XPADJ, XPADJ - ((N ^ (1.0 / (EP - 1))) - 1.0))

# Benchmark

In [20]:
using JuMP

JuMP.delete_lower_bound(get_variable(XQADJ))
JuMP.delete_lower_bound(get_variable(XPADJ))

In [21]:
fix(PY,1)
set_start_value(PX, 1.25)
set_start_value(CX, 1.25)

solve!(M62, cumulative_iteration_limit = 0)
generate_report(M62)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_8FCD.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 15

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,N,1.0,0.0
2,W,1.0,0.0
3,X,1.0,0.0
4,Y,1.0,0.0
5,XI,1.0,0.0
6,PF,1.0,0.0
7,PU,1.0,0.0
8,PW,1.0,0.0
9,CX,1.25,0.0


In [22]:
println("Index: $(value((N*CX^(1-EP))^(1/(1-EP))))")

Index: 1.25


# Counterfactual

In [23]:
set_value!(ENDOW, 2)

solve!(M62)
generate_report(M62)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_902C.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 15

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             3.9901e+02             0.0e+00 (income_balance[CONS)
    1     1     0    15 1.8594e+01  1.0e+00    0.0e+00 (income_balance[CONS)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.8594e+01           I 0.0e+00 1.8e+01 (income_balance[)
    1     1     3     3 2.3207e+00  1.0e+00 SO 0.0e+00 2.1e+00 (income_balance[)
    2     1     4     4 3.5955e-03  1.0e+00 SO 0.0e+00 2.6e-03 (income_balance[)
    3     1     5     5 2.4464e-08  1.0e+00 SO 0.0e+00 2.4e-08 (income_balance[)

Major Iterations. . . . 3
Minor Iteration

Row,var,value,margin
,GenericV…,Float64,Float64
1,N,2.0,-4.66791e-11
2,W,2.18102,-1.1795e-12
3,X,2.0,-1.94712e-10
4,Y,2.0,-2.33356e-10
5,XI,2.0,-1.86688e-10
6,PF,1.0,-3.34985e-10
7,PU,0.917004,3.32182e-9
8,PW,1.0,1.21341e-9
9,CX,1.25,0.0


In [24]:
println("Index: $(value((N*CX^(1-EP))^(1/(1-EP))))")

Index: 1.0511205190574011
